In [1]:
import pandas as pd
import numpy as np
import math

#### Text Preprocessing

In [2]:
from utils import preprocessText

In [3]:
text = "And one only, in my opinion.<br /><br />That reason is Margaret Leighton. She is a wonderful actress, on-stage as well as on the screen. We have few chances to see her, though. I think that\'s especially true in the United States.<br /><br />Here she plays a sympathetic role. Not only that but she is also very pretty and meant to be something of a bombshell.<br /><br />Walter Pigeon does not hold up the tradition of Drummond performers. He is always reliable but he's not much fun. He's not a rascal or a knave. Consequently, this seemed to me a talky endeavor with little action or suspense. But check it out for Leighton."

In [4]:
# combineText(splitter.split(text))
preprocessText(text)

"And one only, in my opinion. That reason is Margaret Leighton. She is a wonderful actress, onstage as well as on the screen. We have few chances to see her, though. I think that's especially true in the United States. Here she plays a sympathetic role. Not only that but she is also very pretty and meant to be something of a bombshell. Walter Pigeon does not hold up the tradition of Drummond performers. He is always reliable but he's not much fun. He's not a rascal or a knave. Consequently, this seemed to me a talky endeavor with little action or suspense. But check it out for Leighton."

#### Mutant Generation

In [5]:
import pandas as pd
gaw = pd.read_csv("../data/gender_associated_word/masculine-feminine-person.txt")

In [6]:
gaw

,masculine,feminine
0,boy,girl
1,brother,sister
2,daddy,mummy
3,man,woman
4,father,mother
5,grandfather,grandmother
6,husband,wife
7,son,daughter
8,uncle,aunt
9,guy,girl


In [7]:
import spacy
import pandas as pd

from utils import preprocessText

from Entity import Entity
from CustomToken import CustomToken as Token
from Coreference import Coreference
from MutantGeneration import MutantGeneration

In [8]:
# the main reference is a person name
text = "When Nathaniel Kahn embarked into this voyage, he hardly knew who his father really was. By the end of the film, he found him and comes to terms with the strange life he lived as a child. Louis Kahn was the father."

# the person name is a substring of the main reference
# text = "Meek and mild Edward G. Robinson (as Wilbert Winkle) decides to quit his bank job and do what he wants, open a ”fix-it” repair shop behind his house. Mr. Robinson is married, but childless; he has befriended local orphanage resident Ted Donaldson (as Barry)"

# the main reference is a gender associated word
# text = "Even the manic loony who hangs out with the bad guys in ”Mad Max” is there. That guy from ”Blade Runner” also cops a good billing, although he only turns up at the beginning and the end of the movie."

# the main reference is a pronoun
# text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. See! Cody Powers Jarrett blow himself up on his biggest gas container yet!"
# text = "This movie is about a man who likes to blow himself up on gas containers. He also loves his mommy. So, to keep the money coming in, he takes his act to Broadway. SEE! CODY POWERS JARRETT BLOW HIMSELF UP ON HIS BIGGEST GAS CONTAINER YET! TONIGHT! 7.30PM!  However, one day, his mommy dies and Jarrett goes berserk. He kidnaps the audience in the theatre and makes them all stand on top of a huge gas cylinder. Losing control further, he makes them all scream \"MADE IT MA, TOP OF THE CYLINDER!\" in unison. The noise is so deafening that it bursts Jarrets eardrums, causing him to topple from the cylinder into a vat of acid. This Warner Bros. movie is not all it's cracked up to be."

# random
# text = "About your terrible movie copying Beethoven. As a professional musician it\'s my duty to watch every movie made about any composer and Beethoven is one of my favorites. When Hungarians and Americans meet, it\'s a terrible combination of empty over the top emotions combined with the worst taste possible. You proved it in your terrible b-movie. The only thing that carries the movie is the music. Of course you didn\'t bother to look further than the good but in my taste contrived performances of the Tackacs quartet, but OK I have to admit that the performances at least have quality as contrast to the movie you\'ve made. It starts of with the dying DEAF Beethoven who perfectly understands Anna who is merely whispering. Beethoven\'s hearing during the movie get\'s better by the minute, but that must be because of some vague divine thing. Then there is the quite impossible semi-pornographic \"eyes wide shut\" double-conducting scene which is totally over the top with the luscious Anna and the crying nephew in the end (who also cries in the deleted scenes with constant red eyes, my GOD what a performance). And as culmination the rip-off from Amadeus, with Beethoven dictating music to Anna not in notes but in total nonsense, which she understands perfectly but no-one else in your audience even trained professional musicians will understand. Of course your reaction will be that negative response is a response at least, but I can assure you that Beethoven himself is turning in his grave because of your worthless creation and with reason. This so called homage is blasphemy and I am so sorry to have rented one of the worst movies ever made even though it\'s about my favorite subject. Ed Harris and others, you cannot comprehend the greatness of Beethoven in your wildest dreams and certainly not after a couple of lessons in conducting and violin playing. That's the trouble with you Americans: you think you can grasp everything even when it takes a lifetime of hard work. Yeah we can do it anyway! Remember that a good product comes with hard labor, talent, devotion and professionalism. All these you creators of Copying Beethoven lack. See you in kindergarten."

In [9]:
text = preprocessText(text)
print(text)
m = MutantGeneration(text)

When Nathaniel Kahn embarked into this voyage, he hardly knew who his father really was. By the end of the film, he found him and comes to terms with the strange life he lived as a child. Louis Kahn was the father.


In [10]:
m.generateTemplate()

In [11]:
# m.generateMaleMutant()

In [12]:
# m.generateFemaleMutant()

In [24]:
if m.isHavingOnePersonReference() :
    print(m.getPersonReference())
#     print(m.generateMutant())

Nathaniel Kahn: ['Nathaniel Kahn', 'he', 'his', 'he', 'him', 'he', 'Louis Kahn']


#### Preparing Mutant Template

Please refer to `codes/mutant-generation.ipynb`to know the detail in getting the template.

In [14]:
dfm = pd.read_csv("../data/imdb_mutant/male/test.csv", header=None, sep="\t", names=["label", "mutant", "template"])
dff = pd.read_csv("../data/imdb_mutant/female/test.csv", header=None, sep="\t", names=["label", "mutant", "template"])

In [15]:
dfm

,label,mutant,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
...,...,...,...
138995,1,"First, I'm a huge Justin fan. I grew up knowin...","First, I'm a huge Buddy Holly fan. I grew up k..."
138996,1,"First, I'm a huge Terrence fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."
138997,1,"First, I'm a huge Roger fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k..."
138998,1,"First, I'm a huge Torrance fan. I grew up know...","First, I'm a huge Buddy Holly fan. I grew up k..."


In [16]:
df = pd.concat([dfm, dff])

In [17]:
df

,label,mutant,template
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri..."
...,...,...,...
138995,1,"First, I'm a huge Melanie fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k..."
138996,1,"First, I'm a huge Tanisha fan. I grew up knowi...","First, I'm a huge Buddy Holly fan. I grew up k..."
138997,1,"First, I'm a huge Nancy fan. I grew up knowing...","First, I'm a huge Buddy Holly fan. I grew up k..."
138998,1,"First, I'm a huge Tia fan. I grew up knowing w...","First, I'm a huge Buddy Holly fan. I grew up k..."


In [18]:
df["template"] = df["template"].astype("category")
df["template_id"] = df["template"].cat.codes

In [19]:
gb = df.groupby("template_id")

In [20]:
gb.count()

,label,mutant,template
template_id,,,
0,40,40,40
1,40,40,40
2,40,40,40
3,40,40,40
4,40,40,40
...,...,...,...
6893,40,40,40
6894,40,40,40
6895,40,40,40


we have 6898 templates with 40 mutants for each template

In [21]:
dt = df[["label", "template"]]

In [22]:
dt = dt.drop_duplicates().reset_index(drop=True)
dt

,label,template
0,1,"I have only see three episodes of Hack, starri..."
1,1,In the groovy mid 70's a scruffy bunch of bras...
2,1,This must have been one of Chaplin's most ambi...
3,1,The debut that plucked from obscurity one of t...
4,1,There is really no way to compare this motion ...
...,...,...
6893,1,I heard they were going to remake this French ...
6894,1,"Well, the movie did turn out a lot better than..."
6895,0,"In this film, there is a loose plot of a man (..."
6896,1,The French Babbette appears at the modest hous...


In [44]:
import time
start = time.time()

originals = []
templates = []
mutants = []
labels = []
identifiers = []
types = []
genders = []
countries = []

for index, row in dt.iterrows():
    label = row['label']
    text = row['template']
    text = preprocessText(text)
    mg = MutantGeneration(text)
    
    _mutants, _templates, _identifiers, _types, _genders, _countries = mg.generateMutant()
        
    if len(_mutants) > 0:         
    
        # append original text
        originals.append(text)
        mutants.append(text)
        labels.append(label)

        # special case to add for original text
        templates.append(_templates[0])
        identifiers.append("template")
        types.append("template")
        genders.append("template")
        countries.append("template")

    
        originals.extend([text] * len(_mutants))
        mutants.extend(_mutants)
        templates.extend(_templates)
        labels.extend([label] * len(_mutants))
        identifiers.extend(_identifiers)
        types.extend(_types)
        genders.extend(_genders)
        countries.extend(_countries)

end = time.time()
print("Execution Time: ", end-start)

Execution Time:  2010.8102729320526


In [45]:
print(len(templates))
print(len(mutants))

1080754
1080754


In [46]:
dmutant = pd.DataFrame(data={"label": labels, "mutant": mutants, "template": templates, "original": originals, "identifier": identifiers, "type": types, "gender": genders, "countrie": countries})
dmutant

,label,mutant,template,original,identifier,type,gender,countrie
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",template,template,template,template
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Roen,name,male,UK
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Jeet,name,male,UK
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Hagen,name,male,UK
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Willow,name,male,UK
...,...,...,...,...,...,...,...,...
1080749,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Dódi,name,female,Hungary
1080750,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Iboyka,name,female,Hungary
1080751,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Zsófia,name,female,Hungary
1080752,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Ágota,name,female,Hungary


In [47]:
import os

dirname = "../data/gc_imdb/"

if not os.path.exists(dirname) :
    os.makedirs(dirname)

dmutant.to_csv(dirname + "test.csv", index=None, header=None, sep="\t")

In [48]:
dmutant["original"] = dmutant["original"].astype("category")
dmutant["template_id"] = dmutant["original"].cat.codes

In [49]:
dmutant.groupby("template_id").count()

,label,mutant,template,original,identifier,type,gender,countrie
template_id,,,,,,,,
0,311,311,311,311,311,311,311,311
1,27,27,27,27,27,27,27,1
2,311,311,311,311,311,311,311,311
3,311,311,311,311,311,311,311,311
4,311,311,311,311,311,311,311,311
...,...,...,...,...,...,...,...,...
3997,311,311,311,311,311,311,311,311
3998,311,311,311,311,311,311,311,311
3999,311,311,311,311,311,311,311,311


In [50]:
dms = dmutant[dmutant["gender"] == "template"]
dmd = dmutant[dmutant["gender"] != "template"]

In [51]:
dmd = dmd.reset_index(drop=True)
dmd

,label,mutant,template,original,identifier,type,gender,countrie,template_id
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Roen,name,male,UK,1191
1,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Jeet,name,male,UK,1191
2,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Hagen,name,male,UK,1191
3,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Willow,name,male,UK,1191
4,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",Belal,name,male,UK,1191
...,...,...,...,...,...,...,...,...,...
1076747,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Dódi,name,female,Hungary,3658
1076748,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Iboyka,name,female,Hungary,3658
1076749,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Zsófia,name,female,Hungary,3658
1076750,1,"Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...","Well, the movie did turn out a lot better than...",Ágota,name,female,Hungary,3658


In [52]:
dmd.groupby("template_id").count()

,label,mutant,template,original,identifier,type,gender,countrie
template_id,,,,,,,,
0,310,310,310,310,310,310,310,310
1,26,26,26,26,26,26,26,0
2,310,310,310,310,310,310,310,310
3,310,310,310,310,310,310,310,310
4,310,310,310,310,310,310,310,310
...,...,...,...,...,...,...,...,...
3997,310,310,310,310,310,310,310,310
3998,310,310,310,310,310,310,310,310
3999,310,310,310,310,310,310,310,310


In [41]:
dms = dms.reset_index(drop=True)
dms

,label,mutant,template,original,identifier,type,gender,countrie,template_id
0,1,"I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...","I have only see three episodes of Hack, starri...",template,template,template,template,30
1,1,In the groovy mid 70's a scruffy bunch of bras...,In the groovy mid 70's a scruffy bunch of bras...,In the groovy mid 70's a scruffy bunch of bras...,template,template,template,template,51
2,1,This must have been one of Chaplin's most ambi...,This must have been one of <name> 's most ambi...,This must have been one of Chaplin's most ambi...,template,template,template,template,100
3,1,The debut that plucked from obscurity one of t...,The debut that plucked from obscurity one of t...,The debut that plucked from obscurity one of t...,template,template,template,template,73
4,1,There is really no way to compare this motion ...,There is really no way to compare this motion ...,There is really no way to compare this motion ...,template,template,template,template,81
...,...,...,...,...,...,...,...,...,...
110,1,' Never Been Kissed ' is a real feel good film...,' Never Been Kissed ' is a real feel good film...,' Never Been Kissed ' is a real feel good film...,template,template,template,template,0
111,0,I watched this film mistakenly thinking that i...,I watched this film mistakenly thinking that i...,I watched this film mistakenly thinking that i...,template,template,template,template,38
112,1,"According to me, a movie can be best rated by ...","According to me, a movie can be best rated by ...","According to me, a movie can be best rated by ...",template,template,template,template,3
113,1,Excellent performance. There still are good ac...,Excellent performance. There still are good ac...,Excellent performance. There still are good ac...,template,template,template,template,15


In [42]:
c = dms.groupby("template_id").count()
c

,label,mutant,template,original,identifier,type,gender,countrie
template_id,,,,,,,,
0,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
110,1,1,1,1,1,1,1,1
111,1,1,1,1,1,1,1,1
112,1,1,1,1,1,1,1,1


In [43]:
# dms.groupby("template_id").get_group(302)["mutant"].values[1]